In [144]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, LeakyReLU
from random import randint
import numpy as np
import os
from PIL import Image
from sklearn.metrics import accuracy_score,confusion_matrix

In [153]:
DRAWING_MAP = {0: "Apple", 1: "Flower", 2: "Cake", 3: "Fish", 4: "Star"}
N_DRAWINGS = 5
N_SAMPLES = 5000
N_EPOCHS = 10
files = ["apple.npy", "flower.npy", "cake.npy", "fish.npy", "star.npy"]

In [154]:
def load(dir, reshaped, files):
    "Takes in a list of filenames and returns a list of numpy arrays."

    data = []
    for file in files:
        f = np.load(dir + file)
        if reshaped:
            new_f = []
            for i in range(len(f)):
                x = np.reshape(f[i], (28, 28))
                x = np.expand_dims(x, axis=0)
                x = np.reshape(f[i], (28, 28, 1))
                new_f.append(x)
            f = new_f
        data.append(f)
    return data


def normalize(data):
    return np.interp(data, [0, 255], [-1, 1])


def denormalize(data):
    return np.interp(data, [-1, 1], [0, 255])


def visualize(array):
    array = np.reshape(array, (28,28))
    img = Image.fromarray(array)
    img.show(title="Visulizing array")


def set_limit(arrays, n):
    new = []
    for array in arrays:
        i = 0
        for item in array:
            if i == n:
                break
            new.append(item)
            i += 1
    return new


def make_labels(N1, N2):
    labels = []
    for i in range(N1):
        labels += [i] * N2
    return labels

In [156]:
drawings = load("Data/", True, files)
drawings = set_limit(drawings, N_SAMPLES)
drawings = list(map(normalize, drawings))
labels = make_labels(N_DRAWINGS, N_SAMPLES)

x_train, x_test, y_train, y_test = tts(drawings, labels, test_size=0.05)
Y_train = keras.utils.to_categorical(y_train, N_DRAWINGS)
Y_test = keras.utils.to_categorical(y_test, N_DRAWINGS)

In [157]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1),padding='same'))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(N_DRAWINGS, activation='softmax'))

In [158]:
model.compile(loss='categorical_crossentropy', 
              optimizer=keras.optimizers.legacy.Adam(learning_rate=0.001), 
              metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)]

In [159]:
model.fit(np.array(x_train), np.array(Y_train), batch_size=32, validation_split=0.3, callbacks=callbacks_list, epochs=N_EPOCHS)

Epoch 1/10
520/520 [==============================] - 6s 11ms/step - loss: 0.2612 - accuracy: 0.9182 - val_loss: 0.1596 - val_accuracy: 0.9547
Epoch 2/10
520/520 [==============================] - 5s 10ms/step - loss: 0.1221 - accuracy: 0.9619 - val_loss: 0.1378 - val_accuracy: 0.9562
Epoch 3/10
520/520 [==============================] - 5s 10ms/step - loss: 0.0903 - accuracy: 0.9706 - val_loss: 0.1262 - val_accuracy: 0.9601
Epoch 4/10
520/520 [==============================] - 5s 10ms/step - loss: 0.0665 - accuracy: 0.9781 - val_loss: 0.1138 - val_accuracy: 0.9639
Epoch 5/10
520/520 [==============================] - 5s 10ms/step - loss: 0.0461 - accuracy: 0.9845 - val_loss: 0.1216 - val_accuracy: 0.9655


In [160]:
y_pred = model.predict(np.array(x_test))

40/40 [==============================] - 0s 3ms/step


In [161]:
score = 0
for i in range(len(y_pred)):
    if np.argmax(y_pred[i]) == y_test[i]:
        score += 1

print("Accuracy: ", ((score + 0.0) / len(y_pred)) * 100)

Accuracy:  96.48


In [162]:
model.save("predictor.h5")

In [75]:
apple = load("Data/", False, ['flower.npy'])

In [79]:
visualize(apple[0][1])